깃허브 참고 링크 : [RHIE-coder](https://github.com/RHIE-coder/Playground.Python/blob/master/%5B03%5DAdvanced/03.%EB%A9%94%ED%83%80%ED%81%B4%EB%9E%98%EC%8A%A4.ipynb)

# 메타 클래스

---

클래스를 만드는 클래스

프레임워크나 모듈을 설계할 때 동적으로 클래스를 만들 수도 있을 것이다.

# 1. type을 사용한 동적 클래스 생성

---

클래스 = type("클래스이름", Tuple:부모클래스, Dict:속성과 메소드)

```python
Calc type('Calculator', (), {})
```

In [5]:
attributes = dict(
  name = "계산기",
  add = lambda self, x, y : x + y,
  mul = lambda self, x, y : x * y
)

Calculator = type("Calculator", (), attributes)

In [6]:
print(Calculator)


<class '__main__.Calculator'>


In [9]:
calc = Calculator()
print(calc.name)
print(calc.add(1, 2))
print(calc.mul(3, 3))

계산기
3
9


## 활용 list 클래스 상속

In [49]:
MyList = type("MyList", (list, ), {
  "__str__" : lambda self : list.__str__(self),
  "odd" : lambda self : [num for num in self if num % 2],
  "even" : lambda self : [num for num in self if not num % 2],
})

In [51]:
mylist = MyList([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(mylist)
print(mylist.odd())
print(mylist.even())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 3, 5, 7, 9]
[2, 4, 6, 8, 10]


# 2. `__new__`

---

지금까지 클래스의 인스턴스를 생성할 때 `__init__`을 통해 생성해봤기 때문에

`__init__` 함수가 인스턴스를 생성하는 함수일 것이라 생각할 수도 있지만,

사실은 `__new__` 함수가 인스턴스를 생성하는 함수다.

또한, `__new__`가 먼저 실행되면 인스턴스는 생성되지만 인스턴스에 대한 정보가 아무것도 없다.

즉, `__new__`는 해당 클래스의 객체(or 인스턴스)를 메모리에 할당을 해주고

클래스의 정보를 객체에 작성해주는 녀석은 `__init__`인 것이다.

In [54]:
class Student:
    def __new__(cls):
        print("CALL __new__()")
        student: Student = object.__new__(cls)
        return student

    def __init__(self):
        print("CALL __init__()")

    def study(self):
        print("공부중")

student = Student()
print(type(student))
student.study()

CALL __new__()
CALL __init__()
<class '__main__.Student'>
공부중


- __new__에 대한 오버라이딩의 목적은 클래스를 커스텀마이징하거나 클래스 생성 등에 대한 핸들링 등이 있다

아래 클래스는 3개의 객체만 생성할 수 있게 제한을 걸어둔 클래스다.

아래와 같은 기능이 필요한 때가 있다.

서버와 데이터베이스를 연결한 connection 객체를 관리해주는 connection pool이라는 객체가 여러 개 있으면 안될 것이다.

그래서 아래와 같은 방법으로 단 1개의 객체만 생성하도록 제한하는 하는 것을 사용한다.

이를 프로그래밍 디자인패턴 기법 중 하나인 싱글톤 패턴(singleton pattern)이라고 한다.

In [59]:
class LimitedInstances():
    __instances = []  # Keep track of instance reference
    limit = 3
    def __new__(cls, *args, **kwargs):
        print("---------인스턴스 생성 전 클래스 정보 확인---------")
        print("| 제한된 객체의 수 : ", cls.limit)
        print("| 현재 생성된 객체 리스트 : ", cls.__instances)
        if not len(cls.__instances) < cls.limit:
            raise RuntimeError("Count not create instance. Limit {} reached".format(cls.limit)) # 강제 에러 발생
        instance = object.__new__(cls, *args, **kwargs)
        cls.__instances.append(instance)
        print("--------------------------------------------------")
        return instance
    
    def __del__(self):
        # Remove instance from __instances
        self.__instance.remove(self)

    @property
    def instances(self):
        return self.__instances

a = LimitedInstances()
LimitedInstances()
LimitedInstances()
LimitedInstances()

---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  []
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x0000023658DE0DF0>]
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x0000023658DE0DF0>, <__main__.LimitedInstances object at 0x0000023658DE0C10>]
--------------------------------------------------
---------인스턴스 생성 전 클래스 정보 확인---------
| 제한된 객체의 수 :  3
| 현재 생성된 객체 리스트 :  [<__main__.LimitedInstances object at 0x0000023658DE0DF0>, <__main__.LimitedInstances object at 0x0000023658DE0C10>, <__main__.LimitedInstances object at 0x0000023658DE0BB0>]


RuntimeError: Count not create instance. Limit 3 reached

In [60]:
a.limit

3

In [61]:
a.instances

# 3. `__call__`

---


`__new__`가 클래스 인스턴스를 생성해 메모리에 할당하고

`__init__`이 인스턴스에 값들을 넣고 초기화하한다면

`__call__`함수는 인스턴스 자체가 호출될 때 호출되는 함수다.

`__new__` ==> `__init__` ==> `__call__`

In [62]:
class MyClass:
    def __init__(self,msg):
        self.__msg = msg
        
    def __call__(self):
        print(len(self.__msg))
        return [s for s in self.__msg]
    
mine = MyClass("Hello World")
mine() # 인스턴스 자체를 호출

11


['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd']

# 4. type을 상속받아 클래스를 만드는 클래스인 메타클래스가 구현하는 방식을 정하기

---

```python
class MetaClassName(type):
    def __new__(metacls, name, bases, namespace):
        #Statement
```

In [63]:
class MakeCalc(type):
    def __new__(matacls, name, bases, namespace):
        namespace["add"] = lambda self, *args : sum(args)
        return type.__new__(matacls, name, bases, namespace)

Calc = MakeCalc("Calculator",(),{})
c = Calc()
print(c.add(1,2,3,4,5,6,7,8,9,10))

55


# 5. 활용 CASE: 싱글톤(Singleton)

---

In [64]:
class Singleton(type):
    __instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs)
        return cls.__instances[cls]
    
class Hello(metaclass=Singleton): # 메타클래스로 Singleton을 지정
    def __init__(self):
        self.desc = "Hello"

a = Hello()
b = Hello()
print(a is b) # 주소가 같음. 같은 객체임

print(a.desc, ":" ,b.desc)
a.desc = "Hello World"
print(a.desc, ":" ,b.desc)

True
Hello : Hello
Hello World : Hello World


- 메타클래스와 new, init, call의 관계를 정리하자면 아래와 같다
- obj == MyClass() == (MyMetaClass())()

In [66]:
class MyMetaClass(type):
    def __new__(cls, *args, **kwargs):
        print("MyMetaClass   __new__")
        return super().__new__(cls,*args,**kwargs)
    
    def __init__(cls, *args, **kwargs):
        print("MyMetaClass   __init__")
        return super().__init__(*args,**kwargs)  
    
    def __call__(cls, *args, **kwargs):
        print("MyMetaClass   __call__")
        return super().__call__(*args,**kwargs)
    
class MyClass(metaclass=MyMetaClass):
    def __new__(cls,*args, **kwargs):
        print('__new__가 호출되었습니다')
        mycls = object.__new__(cls,*args, **kwargs)
        return mycls
    
    def __init__(self):
        print('__init__가 호출되었습니다')
    
    def __call__(self):
        print('__call__가 호출되었습니다')
        
    def hello(self):
        print("hello world")
        
print("="*20)
obj = MyClass()
obj.hello()
obj()

MyMetaClass   __new__
MyMetaClass   __init__
MyMetaClass   __call__
__new__가 호출되었습니다
__init__가 호출되었습니다
hello world
__call__가 호출되었습니다
